In [11]:
import pandas as pd

In [22]:
pd.read_json('data/skin-care-bestsellers-30-products.json', orient='records')

31

In [21]:
pd.read_csv('skin-care-bestsellers-30-reviews.csv', low_memory=False)

,review_id,title,rating,text,date,pos_feedback_count,client_res_count
product_id,,,,,,,
6000016950486,197,180,5,196,192,3,1
6000088767663,62,59,5,62,60,3,1
6000119802279,103,16,5,103,103,5,1
6000188114294,856,688,5,855,856,6,2
6000188115636,4318,1682,5,4314,4150,8,2
6000192034941,5603,2421,5,5598,5602,3,2
6000195508891,19898,1342,5,19881,17680,2,2
6000195953662,140,129,5,138,139,4,2
6000196178609,1006,502,5,1006,1005,2,2
